## Import

In [1]:
from tqdm import tqdm

import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import re


## Load data

In [2]:
data_path = "C:\\Users\\PC0\\Documents\\GitHub\\Korea_Investment_and_Securities\\datasets\\"

In [3]:
am_stocks = pd.read_csv(data_path + "amrican_ticker_stockname.csv")
info_df = pd.read_csv(data_path + "info_df.csv")
tot_df = pd.read_csv(data_path + "tot_df.csv")
final_df = pd.read_csv(data_path + "all_joined_stocks.csv")
etf_niche = pd.read_csv(data_path + "etf_niche_grouped_df.csv")
investing_df = pd.read_csv(data_path + "investing_com.csv")

In [4]:
info_df.rename(columns = {"Unnamed: 0":"Symbol"}, inplace = True)

In [5]:
cap_df = info_df.loc[:, ['Symbol', 'marketCap']]

In [6]:
joined_df = pd.merge(am_stocks, cap_df, how = "inner")

In [15]:
cap_df.head()

,Symbol,marketCap
0,TSM,6.211272e+11
1,V,4.879828e+11
2,JPM,4.489458e+11
3,UNH,4.468141e+11
4,JNJ,4.359586e+11


In [16]:
am_stocks.head()

,Symbol,Name
0,NVDA,NVIDIA Corp
1,QCOM,Qualcomm Inc
2,NXPI,NXP Semiconductors NV
3,AMD,Advanced Micro Devices Inc
4,MRVL,Marvell Technology Inc


In [7]:
joined_df.head()

,Symbol,Name,marketCap
0,NVDA,NVIDIA Corp,6.047336e+11
1,QCOM,Qualcomm Inc,1.849456e+11
2,NXPI,NXP Semiconductors NV,4.943163e+10
3,AMD,Advanced Micro Devices Inc,1.379936e+11
4,MRVL,Marvell Technology Inc,5.716745e+10


In [14]:
investing_df.head()

,etf.com_names,Ticker_names,investing_names,details
0,10x Genomics Inc Class A,None,None,None
1,"17 Education & Technology Group, Inc. Unsponso...",YQ,17 Education Technology Group Inc,Stock - NASDAQ equities
2,1847 Goedeker Inc,None,None,None
3,"1Life Healthcare, Inc.",None,None,None
4,1st Source Corporation,SRCE,1st Source Corporation,Stock - NASDAQ equities


In [30]:
# joined_df.to_csv(data_path + "marketCap_joined_df.csv", index = False)

In [8]:
final_df.drop_duplicates(inplace = True)

In [9]:
# ticker별 최소 길이 stock names 탐색
for idx, ticker in enumerate(final_df.Ticker.unique()):
    temp_df = final_df.loc[final_df.Ticker == ticker]
    len_list = np.array([len(x) for x in temp_df.stock_names])
    min_idx = np.argmin(len_list)
    
    if idx == 0:
        output_df = pd.DataFrame(temp_df.iloc[min_idx,:]).T
    else:
        output_df = output_df.append(pd.DataFrame(temp_df.iloc[min_idx,:]).T)

In [83]:
# output_df.to_csv(data_path + "ticker_stocks_df.csv", index = False)

In [11]:
output_df = pd.read_csv(data_path + "ticker_stocks_df.csv")

In [12]:
temp_df = pd.merge(cap_df, output_df.loc[:, ['Ticker', "stock_names"]], left_on = 'Symbol', right_on = 'Ticker', how = "inner")

del temp_df['Ticker']

temp_df = temp_df.loc[:, ['Symbol', 'stock_names', 'marketCap']]
theme_df = pd.merge(etf_niche, temp_df, on = "stock_names", how = "outer")
theme_df = theme_df.loc[:, ['Theme', 'Symbol', 'stock_names', 'etf_names', 'marketCap']]

In [13]:
theme_df.head()

,Theme,Symbol,stock_names,etf_names,marketCap
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11
1,Agriculture,NVDA,NVIDIA Corporation,1,6.047336e+11
2,Blockchain,NVDA,NVIDIA Corporation,5,6.047336e+11
3,Broad Technology,NVDA,NVIDIA Corporation,3,6.047336e+11
4,Broad Thematic,NVDA,NVIDIA Corporation,11,6.047336e+11


In [105]:
# 최종 데이터 합본
for idx, theme in enumerate(theme_df.Theme.unique()):
    temp_df2 = theme_df.loc[theme_df.Theme == theme, :]
    
    if idx == 0:
        final_df2 = temp_df2.sort_values("etf_names", ascending = False)
    else:
        final_df2 = final_df2.append(temp_df2.sort_values("etf_names", ascending = False))

In [114]:
final_df2.rename(columns = {"etf_names":"cnts"}, inplace = True)
# final_df2.to_csv(data_path + "etf_niche_grouped_df2.csv", index = False)

In [27]:
final_df2 = pd.read_csv(data_path + "etf_niche_grouped_df2.csv")

* ticker 삽입

In [34]:
nan_stocks = final_df2.loc[final_df2.Symbol.isnull(),:].stock_names.unique()

In [35]:
y = "Amazon.com, Inc."

In [40]:
def find_ticker(y):
    global investing_df
    
    try:
        return investing_df.loc[investing_df['etf.com_names'] == y,'Ticker_names'].values[0]
    except:
        return np.nan

In [45]:
find_ticker('VMware, Inc. Class A')

'None'

In [47]:
investing_df.loc[investing_df['etf.com_names'] == "VMware, Inc. Class A"]

,etf.com_names,Ticker_names,investing_names,details
9351,"VMware, Inc. Class A",None,None,None


In [43]:
mapping_dict = {x:find_ticker(x) for x in nan_stocks}

In [58]:
ticker_list = [mapping_dict[x] if mapping_dict[x] != "None" else np.nan for x in final_df2.loc[final_df2.Symbol.isnull(),:].stock_names]

In [63]:
final_df2.loc[final_df2.Symbol.isnull(),'Symbol'] = ticker_list

In [65]:
final_df2.head(10)

,Theme,Symbol,stock_names,cnts,marketCap
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11
1,5G,NXPI,NXP Semiconductors NV,3,4.943163e+10
2,5G,AMD,"Advanced Micro Devices, Inc.",3,1.379936e+11
3,5G,MRVL,"Marvell Technology, Inc.",3,5.716745e+10
4,5G,AMZN,"Amazon.com, Inc.",3,NaN
5,5G,AAPL,Apple Inc.,3,2.770713e+12
6,5G,QCOM,Qualcomm Inc,3,1.849456e+11
7,5G,ADI,"Analog Devices, Inc.",2,NaN
8,5G,MDTKF,MediaTek Inc,2,NaN
9,5G,FB,Meta Platforms Inc. Class A,2,6.055891e+11


In [68]:
final_df3 = pd.merge(final_df2.loc[:, ['Theme', 'Symbol', 'stock_names', 'cnts']], cap_df, on = 'Symbol', how = "left")

In [75]:
select_idx = [False if x or y else True for x,y in zip(final_df3.Symbol.isnull(), final_df3.marketCap.isnull())]

In [77]:
final_df3.loc[select_idx,:].to_csv("matched_df.csv", index = False)

In [81]:
final_df4 = final_df3.loc[select_idx,:]

In [82]:
final_df4.index = [x for x in range(final_df4.shape[0])]

In [83]:
final_df4.head()

,Theme,Symbol,stock_names,cnts,marketCap
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11
1,5G,NXPI,NXP Semiconductors NV,3,4.943163e+10
2,5G,AMD,"Advanced Micro Devices, Inc.",3,1.379936e+11
3,5G,MRVL,"Marvell Technology, Inc.",3,5.716745e+10
4,5G,AMZN,"Amazon.com, Inc.",3,1.573853e+12


* 빈도순 정렬

In [85]:
loop_theme = final_df4.Theme.unique()

In [90]:
for idx, theme in enumerate(loop_theme):
    if idx == 0:
        cnts_df = final_df4.loc[final_df4.Theme == theme].sort_values("cnts", ascending = False)
    else:
        cnts_df = cnts_df.append(final_df4.loc[final_df4.Theme == theme].sort_values("cnts", ascending = False))

In [96]:
cnts_df.to_csv(data_path + "ordered_by_cnts.csv", index = False)

* 시총순 정렬

In [93]:
for idx, theme in enumerate(loop_theme):
    if idx == 0:
        market_order_df = final_df4.loc[final_df4.Theme == theme].sort_values("marketCap", ascending = False)
    else:
        market_order_df = market_order_df.append(final_df4.loc[final_df4.Theme == theme].sort_values("marketCap", ascending = False))

In [97]:
market_order_df.to_csv(data_path + "ordered_by_marketCap.csv", index = False)

* 통합 정렬

In [110]:
cnts_rank = []
caps_rank = []
for idx, theme in enumerate(loop_theme):
    cnts_rank.extend(list(final_df4.loc[final_df4.Theme == theme]['cnts'].rank(ascending=False, method = "min")))
    caps_rank.extend(list(final_df4.loc[final_df4.Theme == theme]['marketCap'].rank(ascending=False, method = "min")))

In [113]:
final_df4['cnts_rank'] = cnts_rank
final_df4['caps_rank'] = caps_rank

<ipython-input-113-2d7ee2b8e57c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['cnts_rank'] = cnts_rank
<ipython-input-113-2d7ee2b8e57c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['caps_rank'] = caps_rank


In [115]:
final_df4['total_rank'] = (final_df4['cnts_rank'] + final_df4['caps_rank'])/2

<ipython-input-115-d31d57bcd899>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df4['total_rank'] = (final_df4['cnts_rank'] + final_df4['caps_rank'])/2


In [120]:
for idx, theme in enumerate(loop_theme):
    if idx == 0:
        tot_order_df = final_df4.loc[final_df4.Theme == theme].sort_values("total_rank")
    else:
        tot_order_df = tot_order_df.append(final_df4.loc[final_df4.Theme == theme].sort_values("total_rank"))

In [121]:
tot_order_df.head()

,Theme,Symbol,stock_names,cnts,marketCap,cnts_rank,caps_rank,total_rank
5,5G,AAPL,Apple Inc.,3,2.770713e+12,2.0,1.0,1.5
4,5G,AMZN,"Amazon.com, Inc.",3,1.573853e+12,2.0,4.0,3.0
0,5G,NVDA,NVIDIA Corporation,4,6.047336e+11,1.0,8.0,4.5
6,5G,QCOM,Qualcomm Inc,3,1.849456e+11,2.0,13.0,7.5
8,5G,FB,Meta Platforms Inc. Class A,2,6.055891e+11,8.0,7.0,7.5


In [122]:
tot_order_df.to_csv(data_path + "ordered_by_mean_ranks.csv", index = False)

* stocks namees.npy 추출

In [173]:
import re

In [233]:
unique_list = final_df2.stock_names.unique()

In [234]:
after_list = [x for x in unique_list if ('-' not in x) and ('%' not in x)]

In [229]:
after_list = [re.sub(" Class A", "", x) for x in np.unique(after_list)]

In [230]:
after_list = [re.sub(" Class B", "", x) for x in np.unique(after_list)]

In [236]:
np.save(data_path + "stock_names.npy",np.unique(after_list))

In [232]:
np.unique(after_list).shape

(10055,)

In [157]:
change_dict = {}

In [159]:
for fdr_name in tot_df.Name:
    empty_list = []
    for etf_name in final_df2.stock_names.unique():
        if fdr_name in etf_name:
            empty_list.append(etf_name)
    change_dict[fdr_name] = empty_list